In [1]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import Hour, Minute
from pandas.tseries.offsets import Day, MonthEnd
from pandas.tseries.offsets import Hour
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


In [2]:


#df = pd.read_csv('Wind_deseason.csv')
df = pd.read_csv('PV_deseason_15.csv', parse_dates=[0], index_col=0)


In [8]:

def model1(tss, scaler, df):
    mse = []

    for train_indx, val_indx in tss.split(df):
        train = df.iloc[train_indx]
        test = df.iloc[val_indx]
        
        
        scaler.fit(train['310_PV_1'].to_frame())
        scaled_train = scaler.transform(train['310_PV_1'].to_frame())
        scaled_test = scaler.transform(test['310_PV_1'].to_frame())
        
        n_input = 3 ##check in book if this is determined with ACF PACF or something else
        n_features = 1 ## see if we should make the neural network with multiple time series (for now just 1)
        generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)
     
        mod = Sequential()
        mod.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
        mod.add(Dense(1))
        mod.compile(optimizer='adam', loss='mse')
       
        
        print(mod.summary())
        
        mod.fit(generator,epochs=50)
        
         
        last_batch = scaled_train[-3:].reshape((1, n_input, n_features))
    
        pred = mod.predict(last_batch)
    
    
        error = 0.5*(test['310_PV_1'][0] - pred[0][0])**2
        mse.append(error)

    print(mse)
    return mse

    

In [9]:
def modelNaive(tss,df):

    mse3 = []

    for train_indx, val_indx in tss.split(df):
        train = df.iloc[train_indx]
        test = df.iloc[val_indx]
    
        pred = train['310_PV_1'][-1]
    
    
        error = 0.5*(test['310_PV_1'][0] - pred)**2
        mse3.append(error)
        
    print(mse3)
    return mse3

In [10]:
def runModels(df):
    tss = TimeSeriesSplit(n_splits = 5, test_size = 20, gap = 0)
    scaler = MinMaxScaler()
    mse1 = model1(tss,scaler,df)
    mse2 = modelNaive(tss,df)
    
    return mse1, mse2

    

In [6]:

df1 = df[df.index.month.isin([1,2,3])]
df2 = df[df.index.month.isin([4,5,6])]
df3 = df[df.index.month.isin([7,8,9])]
df4 = df[df.index.month.isin([10,11,12])]

In [11]:
mse1, mse2 = runModels(df1)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 100)               40800     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
8633/8633 [==============================] - 18s 2ms/step - loss: 0.0069
Epoch 2/50
8633/8633 [==============================] - 17s 2ms/step - loss: 0.0046
Epoch 3/50
8633/8633 [==============================] - 17s 2ms/step - loss: 0.0044
Epoch 4/50
8633/8633 [==============================] - 16s 2ms/step - loss: 0.0044
Epoch 5/50
8633/8633 [==============================] - 18s 2ms/step - loss: 0.0043
Epoch 6/50
8633/8633 

8653/8653 [==============================] - 33s 4ms/step - loss: 0.0039
Epoch 23/50
8653/8653 [==============================] - 28s 3ms/step - loss: 0.0040
Epoch 24/50
8653/8653 [==============================] - 26s 3ms/step - loss: 0.0039
Epoch 25/50
8653/8653 [==============================] - 25s 3ms/step - loss: 0.0039
Epoch 26/50
8653/8653 [==============================] - 26s 3ms/step - loss: 0.0039
Epoch 27/50
8653/8653 [==============================] - 26s 3ms/step - loss: 0.0039
Epoch 28/50
8653/8653 [==============================] - 25s 3ms/step - loss: 0.0039
Epoch 29/50
8653/8653 [==============================] - 26s 3ms/step - loss: 0.0039
Epoch 30/50
8653/8653 [==============================] - 27s 3ms/step - loss: 0.0039
Epoch 31/50
8653/8653 [==============================] - 31s 4ms/step - loss: 0.0039
Epoch 32/50
8653/8653 [==============================] - 25s 3ms/step - loss: 0.0039
Epoch 33/50
8653/8653 [==============================] - 25s 3ms/step - loss:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
mse3, mse4 = runModels(df2)

In [ ]:
mse5, mse6 = runModels(df3)

In [ ]:
mse7, mse8 = runModels(df4)

In [ ]:
print('season 1')

print('model 1:')
print(mse1)
print('model Naive:')
print(mse2)


In [ ]:
print('season 1')

print('model 1:')
print(mse3)
print('model Naive:')
print(mse4)


In [ ]:
print('season 1')

print('model 1:')
print(mse5)
print('model Naive:')
print(mse6)


In [ ]:
print('season 1')

print('model 1:')
print(mse7)
print('model Naive:')
print(mse8)
